# **Predict Relief**

Nesta etapa queremos montar um modelo que, a partir de dados de entrada, prediz se uma reclamação irá incorrer no pagamento de indenização por parte da empresa financeira.

Aqui, nossa variávl alvo é: `Company payed monetary relief` e assume o valor 1 quando a empresa fez o pagamento e 0 quando não fez.

----

# # **Importando Pickle da Base Tratada**

In [10]:
import pandas as pd

df = pd.read_pickle('./pickle/df_processed.pkl')

df.shape

(4812, 138)

In [11]:
df_processed = df.copy()

---

# # **Preparando o Dataset**

## ## **Avaliar as variáveis do dataset**

In [12]:
# Supondo que df seja o nome do seu DataFrame

# Função para garantir que listas sejam convertidas em tuplas antes de contar valores únicos
def count_unique_safe(column):
    try:
        return df[column].apply(lambda x: tuple(x) if isinstance(x, list) else x).nunique()
    except TypeError:
        return 'Not applicable'

# Criando um DataFrame para armazenar as informações das colunas
col_info = pd.DataFrame({
    'Variable Name': df.columns,
    'Data Type': df.dtypes,
    'Unique Values Count': [count_unique_safe(col) for col in df.columns]
})

col_info

,Variable Name,Data Type,Unique Values Count
Date received,Date received,datetime64[ns],1983
Sub-product,Sub-product,object,37
Issue,Issue,object,48
Sub-issue,Sub-issue,object,158
Consumer complaint narrative,Consumer complaint narrative,object,4688
...,...,...,...
Suggested Category,Suggested Category,object,10
Suggested Product/Service,Suggested Product/Service,object,20
Potential Annual Revenue,Potential Annual Revenue,float64,19
Suggested Reasoning,Suggested Reasoning,object,3687


In [19]:
df.columns.to_list()

['Date received Day',
 'Date received Month',
 'Date received Year',
 'Zip life expectancy',
 'Zip average income',
 'Zip average age',
 'Company size_low',
 'Company size_medium',
 'Company market_low',
 'Company market_medium',
 'Company response time_low',
 'Company response time_medium',
 'Company response satisfaction_low',
 'Company response satisfaction_medium',
 'Zip average education_low',
 'Zip average education_medium',
 'Zip bank services access_low',
 'Zip bank services access_medium',
 'Zip crime rate_low',
 'Zip crime rate_medium',
 'Zip unemployment rate_low',
 'Zip unemployment rate_medium',
 'Product_Checking or savings account',
 'Product_Credit card',
 'Product_Credit card or prepaid card',
 'Product_Credit reporting or other personal consumer reports',
 'Product_Credit reporting, credit repair services, or other personal consumer reports',
 'Product_Debt collection',
 'Product_Mortgage',
 'Product_Payday loan, title loan, or personal loan',
 'Product_Prepaid card',

---

## ## **Removendo Colunas Desnecessárias**

- Remover colunas desnecessárias ou que não podem ser usadas para a construção do modelo:
    - `Complaint ID`
    - `Date received`
    - `Sub-product`
    - `Sub-issue`
    - `Issue`
    - `ZIP code`
    - `Consumer complaint narrative`
    - `Company response to consumer`
    
- Remover colunas que possuem poucas informações ou pouca variabilidade:
    - `Tags`

- Remover colunas que indicam informações que só poderíamos ter após a reclamação ser resolvida:
    - `Company response to consumer`
    - `Consumer disputed?`
    - `Suggested Category /Product / Reasoning`
    - `Potential Annual Revenue`

In [13]:
df = df.drop(columns=['Complaint ID'])

df = df.drop(columns=['Company'])

df = df.drop(columns=['Date received'])

df = df.drop(columns=['Sub-product'])

df = df.drop(columns=['Sub-issue'])

df = df.drop(columns=['Issue'])

df = df.drop(columns=['ZIP code'])

df = df.drop(columns=['Consumer complaint narrative'])

df = df.drop(columns=['Company response to consumer'])

df = df.loc[:, ~df.columns.str.startswith('Tags')]

df = df.loc[:, ~df.columns.str.startswith('Company public response_Company')]

df = df.loc[:, ~df.columns.str.startswith('Consumer disputed?')]

df = df.loc[:, ~df.columns.str.startswith('Suggested')]

df = df.drop(columns=['Potential Annual Revenue'])




In [14]:
df.shape

(4812, 108)

---



# # **Modelo de Previsão de Pagamento de Indenização**

- Modelo: **Random Forest Classifier** é um modelo adquado quando a variável alvo é do tipo categórica.

In [20]:
# Importando bibliotecas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.impute import SimpleImputer
import numpy as np

# Expandindo a coluna de embeddings em múltiplas colunas
embedding_df = pd.DataFrame(df['Complaint embedding'].tolist(), index=df.index)

# Concatenando o DataFrame original com os embeddings expandidos
X = pd.concat([df.drop(columns=['Complaint embedding', 'Company payed monetary relief']), embedding_df], axis=1)

# Converter todos os nomes de colunas para strings
X.columns = X.columns.astype(str)

# Preencher valores ausentes com a média das colunas
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

y = df['Company payed monetary relief']  # Variável alvo

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treinando um modelo RandomForest
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Previsões
y_pred = model.predict(X_test)

# Avaliação do modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))


Acurácia: 0.9155124653739612
Matriz de Confusão:
 [[635  87]
 [ 35 687]]
Relatório de Classificação:
               precision    recall  f1-score   support

       False       0.95      0.88      0.91       722
        True       0.89      0.95      0.92       722

    accuracy                           0.92      1444
   macro avg       0.92      0.92      0.92      1444
weighted avg       0.92      0.92      0.92      1444



----

# # **Resultados**

| Metric          | False (Classe 0) | True (Classe 1) | Macro Avg | Weighted Avg |
|-----------------|------------------|-----------------|-----------|--------------|
| **Precision**   | 0.95             | 0.89            | 0.92      | 0.92         |
| **Recall**      | 0.88             | 0.95            | 0.92      | 0.92         |
| **F1-score**    | 0.91             | 0.92            | 0.92      | 0.92         |
| **Support**     | 722              | 722             | 1444      | 1444         |

| **Overall Accuracy** | **0.9155** (91.55%) |


O modelo RandomForest que você treinou produziu resultados bastante sólidos, com uma acurácia de aproximadamente 91,55%. Vamos analisar mais detalhadamente o desempenho com base na matriz de confusão e no relatório de classificação:

### 1. **Acurácia**: 
   - **91,55%**: Isso significa que o modelo classificou corretamente 91,55% das amostras no conjunto de teste.

### 2. **Matriz de Confusão**:
   - **True Negatives (TN)**: 635 amostras foram corretamente classificadas como `False`.
   - **False Positives (FP)**: 87 amostras foram incorretamente classificadas como `True` quando eram `False`.
   - **False Negatives (FN)**: 35 amostras foram incorretamente classificadas como `False` quando eram `True`.
   - **True Positives (TP)**: 687 amostras foram corretamente classificadas como `True`.

### 3. **Relatório de Classificação**:
   - **Para a classe `False`**:
     - **Precision**: 0.95 (95% das previsões de `False` estavam corretas).
     - **Recall**: 0.88 (88% dos verdadeiros `False` foram corretamente identificados).
     - **F1-score**: 0.91 (Média harmônica de precision e recall).
   
   - **Para a classe `True`**:
     - **Precision**: 0.89 (89% das previsões de `True` estavam corretas).
     - **Recall**: 0.95 (95% dos verdadeiros `True` foram corretamente identificados).
     - **F1-score**: 0.92 (Média harmônica de precision e recall).

### 4. **Macro e Weighted Average**:
   - **Macro avg**: Média não ponderada dos valores de precision, recall, e f1-score das duas classes.
   - **Weighted avg**: Média ponderada dos valores de precision, recall, e f1-score, considerando o suporte (número de amostras) de cada classe.

### Interpretação:

- **Precision** é alta para ambas as classes (`False` e `True`), indicando que o modelo tem poucas falsas alarmes (FP) comparado aos acertos (TP).
- **Recall** é ligeiramente maior para a classe `True`, indicando que o modelo está capturando bem a maioria dos casos positivos.
- **F1-score** para ambas as classes está acima de 0.91, o que indica um bom equilíbrio entre precision e recall.
- **Acurácia** geral de 91,55% é muito boa, mas depende do contexto específico do problema se essa taxa é considerada excelente ou apenas satisfatória.

### Possíveis Melhorias:
- **Balanceamento**: Se o conjunto de dados estiver desbalanceado, técnicas como ajuste de pesos, uso de SMOTE (Synthetic Minority Over-sampling Technique), ou balanceamento de classes podem ser exploradas.
- **Otimização de Hiperparâmetros**: O uso de técnicas como Grid Search ou Random Search para otimizar os hiperparâmetros do RandomForest pode melhorar ainda mais o desempenho.
- **Feature Engineering**: Explorar novas variáveis ou transformar as existentes pode ajudar o modelo a capturar padrões mais complexos.

Se precisar de mais ajuda para interpretar esses resultados ou para melhorar o modelo, estou à disposição!